In [2]:
import pandas as pd
import numpy as np
from scipy.sparse import lil_matrix
from sklearn import neighbors
KNN = neighbors.KNeighborsClassifier(metric='euclidean', n_neighbors=1)
from sklearn.model_selection import train_test_split
#Steam ID will be label
#Playhours will be data

In [3]:
user_data = pd.read_csv('ml_cleaned_user_data.csv')

First let's create a list of all the user's steam ideas to act as our labels

In [4]:
steam_ids = list(user_data['Steam_id'])
print(steam_ids[0])
steam_ids = steam_ids[1:418]

76561197960771844


Next let's drop the Steam_ids from the dataset and save the playtime values as an array

In [5]:
ml_data = user_data.drop(['Steam_id'],axis=1).values

Now let's convert this array into a sparse matrix which will get rid of all the 0s. 

In [6]:
test = lil_matrix(ml_data)
sparse_mat = lil_matrix.transpose(test)
sparse_mat

<18726x417 sparse matrix of type '<class 'numpy.float64'>'
	with 84999 stored elements in LInked List format>

Great! But we have the values but they aren't corresponding to any game_id keys so let's save all the column labels into a list so we map the game to the playtime

In [7]:
column_labels = list(user_data.columns)
column_labels = column_labels[1:]

In [8]:
one_user = ml_data[0]

Now we have the Column labels and a sparse matrix we divide up the data randomly and run it through a nearest neighbors algo. Note that we have to transpose the matrix orignally and now again in order for the train test split function not to divide up the rows into two different dataset.

In [9]:
playtime_train0, playtime_test0  = train_test_split(sparse_mat, test_size=0.1, random_state=1)

playtime_train1 = lil_matrix.transpose(playtime_train0)
playtime_train2 = lil_matrix.toarray(playtime_train1)
playtime_train = playtime_train2[1:418]

playtime_test1 = lil_matrix.transpose(playtime_test0)
playtime_test2 = lil_matrix.toarray(playtime_test1)
playtime_test = playtime_test2[1:418]

In [10]:
one_user = playtime_train2[0]
type(one_user)

numpy.ndarray

In [21]:
nn = neighbors.KNeighborsClassifier(n_neighbors=1, metric='euclidean')
nn.fit(playtime_train,steam_ids)
#nn.kneighbors([one_user])
similar_user = nn.predict([one_user])
similar_user

array([76561198353100024], dtype=int64)

In [12]:
similar_user = nn.predict([one_user])
similar_user = str(similar_user[0])
similar_user_data = user_data.query('Steam_id == ' + similar_user )
similar_user_data = similar_user_data.drop(['Steam_id'], axis=1)
user_gameplay_data = user_data.query('Steam_id == 76561197960771844')

for i in (column_labels):    
    one_game_sim_user = int(similar_user_data[str(i)].values[0])
    
    one_game_user = int(user_gameplay_data[str(i)].values[0])
    
    if one_game_sim_user == 0 & one_game_user == 0:
        similar_user_data = similar_user_data.drop([str(i)], axis =1)
        user_gameplay_data = user_gameplay_data.drop([str(i)], axis = 1)
    elif one_game_sim_user == 0 & one_game_user > 0:
        similar_user_data = similar_user_data.drop([str(i)], axis = 1)
        user_gameplay_data = user_gameplay_data.drop([str(i)], axis =1)



'578080'

In [19]:
game_suggestion = 0
for i in similar_user_data:
    one_game_sim_user2 = int(similar_user_data[str(i)].values[0])   
    
    if one_game_sim_user2 > game_suggestion:
        game_suggestion = one_game_sim_user2
        final_suggestion = i 

final_suggestion

'578080'

In [17]:
similar_user_data

,550,570,730,221380,271590,368360,381210,433850,578080
355,438.0,22161.0,86267.0,1593.0,1694.0,136.0,5.0,24964.0,185193.0
